## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Castro,CL,2019-08-26 17:25:49,-42.48,-73.76,48.20,61,40,14.99,scattered clouds,0.000,0
1,1,Lebu,ET,2019-08-26 17:25:49,8.96,38.73,58.69,83,72,1.45,light rain,2.187,0
2,2,Sitka,US,2019-08-26 17:25:49,37.17,-99.65,90.00,46,6,21.00,clear sky,0.000,0
3,3,Torbay,CA,2019-08-26 17:25:49,47.66,-52.73,75.00,43,20,10.29,few clouds,0.000,0
4,4,Grandview,US,2019-08-26 17:25:50,38.89,-94.53,69.80,93,40,4.70,heavy intensity rain,0.000,0


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# min_temp = input("Minimum temperature desired for vacation")
# Max_temp = input("Maximum temperature desired for vacation")

min_temp = 40
max_temp = 100

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
prefer_city_df = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) & (city_data_df["Max Temp"]>=min_temp)]
prefer_city_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Castro,CL,2019-08-26 17:25:49,-42.48,-73.76,48.20,61,40,14.99,scattered clouds,0.000,0
1,1,Lebu,ET,2019-08-26 17:25:49,8.96,38.73,58.69,83,72,1.45,light rain,2.187,0
2,2,Sitka,US,2019-08-26 17:25:49,37.17,-99.65,90.00,46,6,21.00,clear sky,0.000,0
3,3,Torbay,CA,2019-08-26 17:25:49,47.66,-52.73,75.00,43,20,10.29,few clouds,0.000,0
4,4,Grandview,US,2019-08-26 17:25:50,38.89,-94.53,69.80,93,40,4.70,heavy intensity rain,0.000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,560,Dauphin,CA,2019-08-26 17:28:27,51.15,-100.04,68.00,72,90,9.17,overcast clouds,0.000,0
561,561,Deep River,CA,2019-08-26 17:28:27,46.10,-77.49,78.01,60,20,8.03,few clouds,0.000,0
562,562,Port-Gentil,GA,2019-08-26 17:28:27,-0.72,8.78,78.80,83,75,10.29,broken clouds,0.000,0
563,563,Eureka,US,2019-08-26 17:28:27,40.80,-124.16,73.00,68,1,3.36,haze,0.000,0


In [5]:
# 4a. Determine if there are any empty rows.
prefer_city_df.count()

City_ID                     550
City                        550
Country                     545
Date                        550
Lat                         550
Lng                         550
Max Temp                    550
Humidity                    550
Cloudiness                  550
Wind Speed                  550
Current Description         550
Rain inches (last 3 hrs)    550
Snow inches (last 3 hrs)    550
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = prefer_city_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Castro,CL,48.20,scattered clouds,-42.48,-73.76,
1,Lebu,ET,58.69,light rain,8.96,38.73,
2,Sitka,US,90.00,clear sky,37.17,-99.65,
3,Torbay,CA,75.00,few clouds,47.66,-52.73,
4,Grandview,US,69.80,heavy intensity rain,38.89,-94.53,
6,Cidreira,BR,72.01,scattered clouds,-30.17,-50.22,
7,Hilo,US,75.20,heavy intensity rain,19.71,-155.08,
8,Great Falls,US,63.00,clear sky,47.50,-111.29,
9,Lata,IN,41.77,light rain,30.78,78.62,
10,Imeni Stepana Razina,RU,54.91,overcast clouds,54.89,44.30,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()
    print(hotels)
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_dl.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
             print("not found")
        

SyntaxError: unexpected EOF while parsing (<ipython-input-14-26f2ec4e6440>, line 25)

In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [16]:
# 8a. Create the output File (CSV)

# 8b. Export the City_Data into a csv
output_data_file = "test.csv"
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
# 11a. Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))